<a href="https://www.kaggle.com/code/sagorkumarmitra/natural-language-processing-sarcasm-detection?scriptVersionId=145777697" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Dataset

In [25]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2023-09-13 04:22:19--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json.1’

sarcasm.json.1      100%[===================>]   5.38M  6.65MB/s    in 0.8s    

2023-09-13 04:22:20 (6.65 MB/s) - ‘sarcasm.json.1’ saved [5643545/5643545]



In [26]:
import json

with open("./sarcasm.json",'r') as f:
    datastore=json.load(f)
    
sentences=[]
labels=[]

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

# Hyperparameters

In [27]:
training_size=20000
vocab_size=10000

max_length=32
embedding_dim=16

# Split the dataset

In [28]:
training_sentences=sentences[0:training_size]
testing_sentences=sentences[training_size:]

training_labels=labels[0:training_size]
testing_labels=labels[training_size:]

In [38]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok="<OOV>"

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)

tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

training_sequences=tokenizer.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sequences,maxlen=max_length,padding=padding_type, truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,padding=padding_type, truncating=trunc_type)

training_labels=np.array(training_labels)
testing_labels=np.array(testing_labels)

# Model

In [30]:
import tensorflow as tf

gap1d_layer=tf.keras.layers.GlobalAveragePooling1D()
sample_array=np.array([[[10,2],[1,3],[1,1]]])
output = gap1d_layer(sample_array)

In [31]:
import numpy as np
np.array([[10,2],[1,3],[1,1]])

array([[10,  2],
       [ 1,  3],
       [ 1,  1]])

In [32]:
np.array([[[10,2],[1,3],[1,1]]])

array([[[10,  2],
        [ 1,  3],
        [ 1,  1]]])

In [33]:
output.shape

TensorShape([1, 2])

In [34]:
output.numpy()

array([[4, 2]])

In [35]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 32, 16)            160000    
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
num_epochs = 30

# Train the model
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
# Plot the accuracy and loss
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")